In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
subject = 'Paraná - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [9]:
data = pd.read_csv('2003_01_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - Consumo de Cimento (t)
0,2003,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.284196,3.647166,4018.882255,2.027147e+08,1.060612e+07,17.589536,1.851788e+08,0.776871,2158.499000
1,2004,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.209520,3.582935,4010.031842,2.055103e+08,1.072430e+07,17.625670,1.858242e+08,0.778118,2259.910000
2,2005,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.186736,3.430058,4028.768630,2.080841e+08,1.084112e+07,17.657353,1.863343e+08,0.779266,2225.422000
3,2006,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.093445,3.364859,4330.599545,2.106490e+08,1.096989e+07,17.713982,1.870506e+08,0.780317,2203.646000
4,2007,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.065900,3.446867,4749.071195,2.130490e+08,1.112092e+07,17.772465,1.877383e+08,0.781274,2382.720000
5,2008,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.033948,3.612886,5058.377260,2.148670e+08,1.122835e+07,17.780948,1.878837e+08,0.782138,3001.521000
6,2009,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.063370,3.874729,5346.288081,2.163260e+08,1.133139e+07,17.787925,1.879466e+08,0.782910,3092.068000
7,2010,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.026843,4.308613,5527.794738,2.218671e+08,1.138377e+07,17.886000,1.893914e+08,0.783594,3749.448000
8,2011,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.013806,4.707700,5623.811148,2.270561e+08,1.138563e+07,17.967281,1.906364e+08,0.784190,4174.942000
9,2012,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,6.472496,5.027391,5779.044240,2.317175e+08,1.133915e+07,18.011636,1.916091e+08,0.784389,4522.969000


In [10]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.284196,3.647166,4018.882255,2.027147e+08,1.060612e+07,17.589536,1.851788e+08,0.776871
1,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.209520,3.582935,4010.031842,2.055103e+08,1.072430e+07,17.625670,1.858242e+08,0.778118
2,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.186736,3.430058,4028.768630,2.080841e+08,1.084112e+07,17.657353,1.863343e+08,0.779266
3,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.093445,3.364859,4330.599545,2.106490e+08,1.096989e+07,17.713982,1.870506e+08,0.780317
4,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.065900,3.446867,4749.071195,2.130490e+08,1.112092e+07,17.772465,1.877383e+08,0.781274
5,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.033948,3.612886,5058.377260,2.148670e+08,1.122835e+07,17.780948,1.878837e+08,0.782138
6,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.063370,3.874729,5346.288081,2.163260e+08,1.133139e+07,17.787925,1.879466e+08,0.782910
7,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.026843,4.308613,5527.794738,2.218671e+08,1.138377e+07,17.886000,1.893914e+08,0.783594
8,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.013806,4.707700,5623.811148,2.270561e+08,1.138563e+07,17.967281,1.906364e+08,0.784190
9,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,6.472496,5.027391,5779.044240,2.317175e+08,1.133915e+07,18.011636,1.916091e+08,0.784389


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2259.910000
1     2225.422000
2     2203.646000
3     2382.720000
4     3001.521000
5     3092.068000
6     3749.448000
7     4174.942000
8     4522.969000
9     4578.716000
10    4341.821857
11    4108.786286
12    3869.334857
13    3764.518000
14    3773.354000
15    3786.974000
16    4235.532000
17    4784.413000
18    4538.256000
19            NaN
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,1.109126,-1.002190,-1.667898,-1.603182,-2.155349,-1.836009,-1.727113,-2.028667
1,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,0.950765,-1.087231,-1.679381,-1.407610,-1.617234,-1.606661,-1.487031,-1.640040
2,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,0.902449,-1.289637,-1.655070,-1.227554,-1.085281,-1.405566,-1.297277,-1.282141
3,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,0.704613,-1.375961,-1.263451,-1.048118,-0.498915,-1.046128,-1.030811,-0.954387
4,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,0.646198,-1.267383,-0.720493,-0.880219,0.188846,-0.674928,-0.774984,-0.656193
5,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,0.578439,-1.047577,-0.319175,-0.753036,0.678037,-0.621083,-0.720881,-0.386975
6,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,0.640834,-0.700901,0.054383,-0.650968,1.147222,-0.576797,-0.697501,-0.146150
7,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.563373,-0.126446,0.289884,-0.263323,1.385758,0.045699,-0.160026,0.066866
8,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.535726,0.401938,0.414463,0.099685,1.394217,0.561610,0.303092,0.252658
9,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,-2.732837,0.825204,0.615874,0.425785,1.182549,0.843135,0.664950,0.314660


In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     2259.910000
1     2225.422000
2     2203.646000
3     2382.720000
4     3001.521000
5     3092.068000
6     3749.448000
7     4174.942000
8     4522.969000
9     4578.716000
10    4341.821857
11    4108.786286
12    3869.334857
13    3764.518000
14    3773.354000
15    3786.974000
16    4235.532000
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [14]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-1.059306,1.648055,1.965957,0.815772,-0.261923,0.400377,0.581416,-0.093694


In [15]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    4784.413
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [16]:
train_input.iloc[-(len(train_input)//5):]

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
14,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,-0.441264,0.516338,0.775682,1.201787,-0.478243,0.912412,1.048657,1.309955
15,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,-0.680946,0.682550,1.013605,1.168595,-0.457848,0.842867,0.969305,1.043055
16,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,-0.832798,1.136575,1.594449,1.062470,-0.378078,0.681669,0.840429,0.525187


In [17]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [21]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2743900119, 3965451349, 4001752518, 2703870979, 2270702476, 3960318116, 3545769499, 2802717035, 2190081861, 2403240284, 1066831723, 2168933867, 3526730839, 3998998149, 3169983917, 562475291, 2461888607, 2496048751, 1889803535, 3673860386, 914894861, 3186843473, 3116198715, 4052400734, 96921522, 448915994, 1339928978, 279691596, 3215288423, 4096286657, 628909430, 3266775801, 1810932025, 1972609249, 2270467465, 1504946090, 2645409800, 2574337506, 1637131858, 3173406514, 1043758756, 695392576, 1385810782, 3341485654, 592804838, 3818540856, 2081118055, 2196651422, 38639629, 861077619, 3367236874, 3310575422, 1385861546, 2262013766, 4056536252, 342258075, 2696430252, 4248966344, 1030926027, 144763947, 719337078, 24213183, 3060063531, 3340940802, 3501800035, 3935392707, 3707425997, 513914892, 1727524470, 2047278314, 4048548516, 2159740943, 1688483365, 3909610423, 2706426362, 3259733402, 1440855663, 1197619400, 2896899097, 2593655329, 344344196, 2290245432, 706430749, 3179035611, 3654584850,

val_loss: 30536.1640625


Step: 88 ___________________________________________
val_loss: 46111.01953125


Step: 89 ___________________________________________
val_loss: 46110.98046875


Step: 90 ___________________________________________
val_loss: 46111.96875


Step: 91 ___________________________________________
val_loss: 21384.5078125


Step: 92 ___________________________________________
val_loss: 47059.67578125


Step: 93 ___________________________________________
val_loss: 46112.51953125


Step: 94 ___________________________________________
val_loss: 46170.125


Step: 95 ___________________________________________
val_loss: 37763.890625


Step: 96 ___________________________________________
val_loss: 52573.80859375


Step: 97 ___________________________________________
val_loss: 47118.38671875


Step: 98 ___________________________________________
val_loss: 46201.1875


Step: 99 ___________________________________________
val_loss: 45021.234375


final_seed: 3960318116


In [22]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 440ms/step - loss: 13145810.0000 - val_loss: 15257457.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 13007600.0000 - val_loss: 8255690.5000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 11834316.0000 - val_loss: 2232905.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 6279410.0000 - val_loss: 4580370.5000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 6998611.0000 - val_loss: 28754214.0000
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 16176375.0000 - val_loss: 7164512.5000
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 9452857.0000 - val_loss: 3827253.2500
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 8681587.0000 - val_loss: 3425929.2500
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 8396312.0000 - val_loss

1/1 [==============================] - 0s 13ms/step - loss: 1864415.7500 - val_loss: 2084402.0000
Epoch 74/10000
1/1 [==============================] - 0s 13ms/step - loss: 3227998.7500 - val_loss: 3399544.7500
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 2569152.7500 - val_loss: 1871324.0000
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 2639505.5000 - val_loss: 2656470.0000
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 3018039.5000 - val_loss: 2320311.2500
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 2394485.7500 - val_loss: 1522063.8750
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 2661726.5000 - val_loss: 1850869.3750
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 2331767.7500 - val_loss: 1710000.1250
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 2536792.7500 - val_loss: 2024046.500

1/1 [==============================] - 0s 13ms/step - loss: 942593.9375 - val_loss: 476122.0000
Epoch 146/10000
1/1 [==============================] - 0s 13ms/step - loss: 783920.6875 - val_loss: 334308.3438
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 618970.1250 - val_loss: 286246.6562
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 728449.7500 - val_loss: 423820.6250
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 954414.3125 - val_loss: 510229.5938
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 680204.0000 - val_loss: 272466.9688
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 889715.9375 - val_loss: 452875.6562
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 778234.6875 - val_loss: 489170.9688
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 541043.2500 - val_loss: 237126.9844
Epoch 15

1/1 [==============================] - 0s 13ms/step - loss: 197735.1562 - val_loss: 68549.5703
Epoch 219/10000
1/1 [==============================] - 0s 14ms/step - loss: 469821.9062 - val_loss: 176644.1719
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 252321.2344 - val_loss: 78147.4375
Epoch 221/10000
1/1 [==============================] - 0s 13ms/step - loss: 331313.7188 - val_loss: 144486.8125
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 366647.6250 - val_loss: 81742.2500
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 300118.2188 - val_loss: 117987.2188
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 655087.6250 - val_loss: 135689.0469
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 368018.0625 - val_loss: 72331.9688
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 346969.2500 - val_loss: 105447.3516
Epoch 227/10

Epoch 292/10000
1/1 [==============================] - 0s 13ms/step - loss: 149521.4844 - val_loss: 48956.9648
Epoch 293/10000
1/1 [==============================] - 0s 13ms/step - loss: 162224.3438 - val_loss: 46110.9648
Epoch 294/10000
1/1 [==============================] - 0s 13ms/step - loss: 181503.1719 - val_loss: 50192.5117
Epoch 295/10000
1/1 [==============================] - 0s 13ms/step - loss: 301443.3438 - val_loss: 90747.5547
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 138241.1719 - val_loss: 47181.0000
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 282438.5938 - val_loss: 68760.4219
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 346987.3438 - val_loss: 79978.7578
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 382038.3438 - val_loss: 90426.6875
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 147723.3281 - val_loss: 46915.3789
E

...

Epoch 736/10000
1/1 [==============================] - 0s 13ms/step - loss: 248805.0938 - val_loss: 46655.4258
Epoch 737/10000
1/1 [==============================] - 0s 13ms/step - loss: 371183.9375 - val_loss: 47282.3867
Epoch 738/10000
1/1 [==============================] - 0s 13ms/step - loss: 150789.3281 - val_loss: 94520.0547
Epoch 739/10000
1/1 [==============================] - 0s 14ms/step - loss: 231689.5156 - val_loss: 47609.6562
Epoch 740/10000
1/1 [==============================] - 0s 13ms/step - loss: 234891.3750 - val_loss: 47606.4414
Epoch 741/10000
1/1 [==============================] - 0s 14ms/step - loss: 199248.9062 - val_loss: 59356.7773
Epoch 742/10000
1/1 [==============================] - 0s 13ms/step - loss: 244993.7031 - val_loss: 46706.5781
Epoch 743/10000
1/1 [==============================] - 0s 14ms/step - loss: 249419.0312 - val_loss: 52947.0742
Epoch 744/10000
1/1 [==============================] - 0s 14ms/step - loss: 285024.6250 - val_loss: 46639.5312
E

Epoch 810/10000
1/1 [==============================] - 0s 13ms/step - loss: 245410.3281 - val_loss: 54885.3125
Epoch 811/10000
1/1 [==============================] - 0s 13ms/step - loss: 171247.7031 - val_loss: 61564.7773
Epoch 812/10000
1/1 [==============================] - 0s 13ms/step - loss: 308326.6562 - val_loss: 53701.0664
Epoch 813/10000
1/1 [==============================] - 0s 13ms/step - loss: 176427.5000 - val_loss: 57689.9258
Epoch 814/10000
1/1 [==============================] - 0s 13ms/step - loss: 172180.2500 - val_loss: 67197.3594
Epoch 815/10000
1/1 [==============================] - 0s 13ms/step - loss: 207796.7188 - val_loss: 49191.9258
Epoch 816/10000
1/1 [==============================] - 0s 13ms/step - loss: 231328.3125 - val_loss: 46337.7539
Epoch 817/10000
1/1 [==============================] - 0s 13ms/step - loss: 129306.0859 - val_loss: 86614.7422
Epoch 818/10000
1/1 [==============================] - 0s 13ms/step - loss: 447937.3125 - val_loss: 47518.2930
E

Epoch 884/10000
1/1 [==============================] - 0s 13ms/step - loss: 244011.4688 - val_loss: 51799.5820
Epoch 885/10000
1/1 [==============================] - 0s 13ms/step - loss: 166787.9375 - val_loss: 67160.7891
Epoch 886/10000
1/1 [==============================] - 0s 13ms/step - loss: 284129.2500 - val_loss: 49806.2227
Epoch 887/10000
1/1 [==============================] - 0s 13ms/step - loss: 133629.3750 - val_loss: 82534.7812
Epoch 888/10000
1/1 [==============================] - 0s 13ms/step - loss: 294831.1250 - val_loss: 46989.8477
Epoch 889/10000
1/1 [==============================] - 0s 13ms/step - loss: 257600.5312 - val_loss: 77062.2734
Epoch 890/10000
1/1 [==============================] - 0s 14ms/step - loss: 268624.8438 - val_loss: 58296.3789
Epoch 891/10000
1/1 [==============================] - 0s 13ms/step - loss: 377432.2500 - val_loss: 46129.5156
Epoch 892/10000
1/1 [==============================] - 0s 13ms/step - loss: 217149.5469 - val_loss: 52201.6875
E

Epoch 958/10000
1/1 [==============================] - 0s 13ms/step - loss: 217536.5000 - val_loss: 71907.3203
Epoch 959/10000
1/1 [==============================] - 0s 14ms/step - loss: 195827.0469 - val_loss: 82801.6016
Epoch 960/10000
1/1 [==============================] - 0s 13ms/step - loss: 137437.0625 - val_loss: 78052.6250
Epoch 961/10000
1/1 [==============================] - 0s 14ms/step - loss: 160267.7344 - val_loss: 58621.1094
Epoch 962/10000
1/1 [==============================] - 0s 14ms/step - loss: 156060.7656 - val_loss: 87100.4609
Epoch 963/10000
1/1 [==============================] - 0s 14ms/step - loss: 156832.2812 - val_loss: 73996.1016
Epoch 964/10000
1/1 [==============================] - 0s 14ms/step - loss: 176816.7969 - val_loss: 84622.2109
Epoch 965/10000
1/1 [==============================] - 0s 14ms/step - loss: 192019.8281 - val_loss: 68052.7422
Epoch 966/10000
1/1 [==============================] - 0s 14ms/step - loss: 210691.7812 - val_loss: 91055.2266
E

1/1 [==============================] - 0s 13ms/step - loss: 138932.3906 - val_loss: 57492.1680
Epoch 1032/10000
1/1 [==============================] - 0s 13ms/step - loss: 135803.5156 - val_loss: 59938.4805
Epoch 1033/10000
1/1 [==============================] - 0s 13ms/step - loss: 125308.2031 - val_loss: 67442.7578
Epoch 1034/10000
1/1 [==============================] - 0s 14ms/step - loss: 130904.6016 - val_loss: 68445.1953
Epoch 1035/10000
1/1 [==============================] - 0s 13ms/step - loss: 125487.1328 - val_loss: 75569.1641
Epoch 1036/10000
1/1 [==============================] - 0s 13ms/step - loss: 111148.7734 - val_loss: 105125.8672
Epoch 1037/10000
1/1 [==============================] - 0s 13ms/step - loss: 107457.2734 - val_loss: 71274.2891
Epoch 1038/10000
1/1 [==============================] - 0s 13ms/step - loss: 100373.2500 - val_loss: 90423.1250
Epoch 1039/10000
1/1 [==============================] - 0s 13ms/step - loss: 112744.5469 - val_loss: 99345.1016
Epoch 10

In [23]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[4784.413](test_target) - [[4192.209]](prediction) = 592.2040156249996


In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.854322,-0.039492,-1.053361,-1.535732,-1.563966,-1.486447,-1.569376,-1.670526
1,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,1.008433,-0.263029,-1.068524,-1.054354,-1.123510,-1.094963,-1.060326,-1.106095
2,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.750349,-0.795068,-1.036424,-0.611167,-0.688099,-0.751705,-0.657986,-0.586293
3,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.306402,-1.021975,-0.519322,-0.169507,-0.208149,-0.138167,-0.092993,-0.110272
4,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.618426,-0.736570,0.197610,0.243755,0.354794,0.495450,0.449443,0.322816
5,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.980361,-0.158794,0.727518,0.556802,0.755205,0.587361,0.564158,0.713819
6,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.647075,0.752467,1.220771,0.808031,1.139240,0.662954,0.613731,1.063586
7,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-1.060840,2.262462,1.531732,1.762172,1.334485,1.725517,1.753350,1.372965


train_target:


,Paraná - Consumo de Cimento (t)
0,2259.910
1,2225.422
2,2203.646
3,2382.720
4,3001.521
5,3092.068
6,3749.448
7,4174.942


test_input:


...

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - Valor Cimento Portland,Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-1.059306,1.648055,1.965957,0.815772,-0.261923,0.400377,0.581416,-0.093694


test_target:


,Paraná - Consumo de Cimento (t)
17,4784.413


1/1 [==============================] - 0s 34ms/step
Error: 592.2040156249996




[3999.956787109375,
 4201.5576171875,
 4433.0185546875,
 4051.3642578125,
 4225.4814453125,
 4360.1591796875,
 3925.016845703125,
 3798.4443359375,
 3841.585693359375,
 4192.208984375]

In [26]:
display(targets)
display(predictions)

[4522.969,
 4578.716,
 4341.821857142857,
 4108.786285714286,
 3869.334857142857,
 3764.518,
 3773.354,
 3786.974,
 4235.532,
 4784.413]

[3999.956787109375,
 4201.5576171875,
 4433.0185546875,
 4051.3642578125,
 4225.4814453125,
 4360.1591796875,
 3925.016845703125,
 3798.4443359375,
 3841.585693359375,
 4192.208984375]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

314.98605929129474

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.07541610385398248